<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Data-files" data-toc-modified-id="Data-files-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data files</a></span><ul class="toc-item"><li><span><a href="#Metadata-for-the-database" data-toc-modified-id="Metadata-for-the-database-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Metadata for the database</a></span></li><li><span><a href="#Setting-the-file-paths" data-toc-modified-id="Setting-the-file-paths-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Setting the file paths</a></span></li><li><span><a href="#How-to-load-and-read-a-json-file" data-toc-modified-id="How-to-load-and-read-a-json-file-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>How to load and read a json file</a></span></li></ul></li><li><span><a href="#Data-Cleaning" data-toc-modified-id="Data-Cleaning-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data Cleaning</a></span></li><li><span><a href="#Data-Preprocessing" data-toc-modified-id="Data-Preprocessing-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data Preprocessing</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#TODO:-Clean-first-word-dirty-data" data-toc-modified-id="TODO:-Clean-first-word-dirty-data-4.0.1"><span class="toc-item-num">4.0.1&nbsp;&nbsp;</span>TODO: Clean first word dirty data</a></span></li><li><span><a href="#END-TODO" data-toc-modified-id="END-TODO-4.0.2"><span class="toc-item-num">4.0.2&nbsp;&nbsp;</span>END TODO</a></span></li></ul></li><li><span><a href="#Forming-n-grams" data-toc-modified-id="Forming-n-grams-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Forming n-grams</a></span></li></ul></li><li><span><a href="#Checking-the-Vocabulary-of-our-dataset" data-toc-modified-id="Checking-the-Vocabulary-of-our-dataset-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Checking the Vocabulary of our dataset</a></span></li><li><span><a href="#Word-Cloud-of-Abstracts" data-toc-modified-id="Word-Cloud-of-Abstracts-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Word Cloud of Abstracts</a></span></li><li><span><a href="#Forming-word-Vectors" data-toc-modified-id="Forming-word-Vectors-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Forming word Vectors</a></span></li><li><span><a href="#Dimensionality-Reduction" data-toc-modified-id="Dimensionality-Reduction-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Dimensionality Reduction</a></span></li><li><span><a href="#Clustering" data-toc-modified-id="Clustering-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Clustering</a></span></li><li><span><a href="#Visualization-and-stuff" data-toc-modified-id="Visualization-and-stuff-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Visualization and stuff</a></span></li><li><span><a href="#References" data-toc-modified-id="References-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>References</a></span></li></ul></div>

# Goal

There is a need to respond quickly to changes in today's world. This response may sometimes require understanding of a topic. Given any topic, there is a plethora of sources providing seas of data related to the it. Given todays world condition, a global concern is the treatment and handling of the COVID-19 pandemic. Given the rising amount of literature regarding the subject, it will be difficult for a scientist to keep up with today's literature.
> Is it possible to cluster these tens of thousands of articles to make it easier for a scientist to find relevant research articles?



In [7]:
from datetime import datetime
start_time = datetime.now()
import pandas as pd
import os
import json
import glob
import matplotlib.pyplot as plt
from hdfs import InsecureClient
end_time = datetime.now()
print(f'Time for imports only: {end_time-start_time}')

Time for imports only: 0:00:01.366413


In [8]:
root_path = "/s/chopin/b/grad/sanketm/cord19dataset/"

In [9]:
os.chdir(root_path)
os.listdir()


['noncomm_use_subset',
 'cord19_specter_embeddings_2020-04-10',
 'custom_license',
 'biorxiv_medrxiv',
 'COVID.DATA.LIC.AGMT.pdf',
 'metadata.readme',
 'json_schema.txt',
 'comm_use_subset',
 'biorxiv_medrxiv.tar.gz',
 'metadata.csv',
 'images']

# Data files

The data files that we have been given are of the format that follows:

![title](/s/chopin/b/grad/sanketm/cord19dataset/images/article_format.png)

## Metadata for the database

We are given a metadata file that gives us the filename of each article along with its abstract and some other details such as date published and journal that it was published in.

In [10]:
meta = pd.read_csv("./metadata.csv")
meta.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,xqhn0vbp,1e1286db212100993d03cc22374b624f7caee956,PMC,Airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,PMC140314,12525263.0,no-cc,"BACKGROUND: Rhinovirus, the most common cause ...",2003-01-13,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",BMC Public Health,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
1,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,PMC,Discovering human history from stomach bacteria,10.1186/gb-2003-4-5-213,PMC156578,12734001.0,no-cc,Recent analyses of human pathogens have reveal...,2003-04-28,"Disotell, Todd R",Genome Biol,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
2,le0ogx1s,NaN,PMC,A new recruit for the army of the men of death,10.1186/gb-2003-4-7-113,PMC193621,12844350.0,no-cc,"The army of the men of death, in John Bunyan's...",2003-06-27,"Petsko, Gregory A",Genome Biol,NaN,NaN,False,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
3,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,PMC,Association of HLA class I with severe acute r...,10.1186/1471-2350-4-9,PMC212558,12969506.0,no-cc,BACKGROUND: The human leukocyte antigen (HLA) ...,2003-09-12,"Lin, Marie; Tseng, Hsiang-Kuang; Trejaut, Jean...",BMC Med Genet,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
4,0qaoam29,5b68a553a7cbbea13472721cd1ad617d42b40c26,PMC,A double epidemic model for the SARS propagation,10.1186/1471-2334-3-19,PMC222908,12964944.0,no-cc,BACKGROUND: An epidemic of a Severe Acute Resp...,2003-09-10,"Ng, Tuen Wai; Turinici, Gabriel; Danchin, Antoine",BMC Infect Dis,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...


In [11]:
meta.count()

cord_uid                       51078
sha                            38022
source_x                       51078
title                          50920
doi                            47741
pmcid                          41082
pubmed_id                      37861
license                        51078
abstract                       42352
publish_time                   51070
authors                        48891
journal                        46368
Microsoft Academic Paper ID      964
WHO #Covidence                  1768
has_pdf_parse                  51078
has_pmc_xml_parse              51078
full_text_file                 42511
url                            50776
dtype: int64

In [12]:
meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51078 entries, 0 to 51077
Data columns (total 18 columns):
cord_uid                       51078 non-null object
sha                            38022 non-null object
source_x                       51078 non-null object
title                          50920 non-null object
doi                            47741 non-null object
pmcid                          41082 non-null object
pubmed_id                      37861 non-null float64
license                        51078 non-null object
abstract                       42352 non-null object
publish_time                   51070 non-null object
authors                        48891 non-null object
journal                        46368 non-null object
Microsoft Academic Paper ID    964 non-null float64
WHO #Covidence                 1768 non-null object
has_pdf_parse                  51078 non-null bool
has_pmc_xml_parse              51078 non-null bool
full_text_file                 42511 non-null ob

Let us look at the column marked "SHA". At a closer look, we can see that it seems to be a hash of some kind.

But looking into the folders, we see that this unique hash has been ingeniously used to identify each file. i.e. We don't need to refer to a long paper title to find its file contents. Instead we can just scan the home directory for a file with that hash.json.

In [13]:
meta["sha"][0]

'1e1286db212100993d03cc22374b624f7caee956'

In [14]:
bio= meta.loc[meta['source_x'] == 'biorxiv']
bio.head()
bio.iloc[1]['sha']

'daf32e013d325a6feb80e83d15aabc64a48fae33'

Now the source_x column is of importance for us as well. It refers to the source journal of each paper. This is important because the papers have been organized into folders, each representing a different source. e.g. a biorxiv folder, a WHO document folder etc.

In [15]:
meta.source_x.unique()

array(['PMC', 'Elsevier', 'CZI', 'WHO', 'biorxiv', 'medrxiv'],
      dtype=object)

## Setting the file paths

As we can see, there are up to six different sources whose documents we have been provided. We can try to make the document search easy by trying to make a dictionary to map sources with folder paths to search for each row of the document.

In [16]:
file_path = {}
file_path.fromkeys(meta.source_x.unique())

{'PMC': None,
 'Elsevier': None,
 'CZI': None,
 'WHO': None,
 'biorxiv': None,
 'medrxiv': None}

In [17]:
file_path['biorxiv'] = "/s/chopin/b/grad/sanketm/cord19dataset/biorxiv_medrxiv/biorxiv_medrxiv/pdf_json/"


In [18]:
os.chdir(file_path['biorxiv'])

Let us read some of the biorxiv files into variables to see the structure as viewed in the screenshot before section 2.1.

In [19]:
article_array = {}
for i in range(5):
    article_array[i] = json.load(open(bio.iloc[i]['sha'] + ".json"))


In [20]:
for i in range(5):
    print(article_array[i])

{'paper_id': 'f056da9c64fbf00a4645ae326e8a4339d015d155', 'metadata': {'title': 'SIANN: Strain Identification by Alignment to Near Neighbors', 'authors': [{'first': 'Samuel', 'middle': ['S'], 'last': 'Minot', 'suffix': '', 'affiliation': {}, 'email': ''}, {'first': 'Stephen', 'middle': ['D'], 'last': 'Turner', 'suffix': '', 'affiliation': {}, 'email': ''}, {'first': 'Krista', 'middle': ['L'], 'last': 'Ternus', 'suffix': '', 'affiliation': {}, 'email': ''}, {'first': 'Dana', 'middle': ['R'], 'last': 'Kadavy', 'suffix': '', 'affiliation': {}, 'email': ''}]}, 'abstract': [{'text': 'Next-generation sequencing is increasingly being used to study samples composed of mixtures of organisms, such as in clinical applications where the presence of a pathogen at very low abundance may be highly important. We present an analytical method (SIANN: Strain Identification by Alignment to Near Neighbors) specifically designed to rapidly detect a set of target organisms in mixed samples that achieves a hig

In [21]:
article_array[0].keys()

dict_keys(['paper_id', 'metadata', 'abstract', 'body_text', 'bib_entries', 'ref_entries', 'back_matter'])

From above, we can see that there are keys that describe each paper of the biorxiv journal. Of these, Our paritcular interest lies in the abstract and the body_text. The title is useful too, and to get it, we have to look inside the metadata part of the file. 

In [22]:
article_array[0]['metadata'].keys()

dict_keys(['title', 'authors'])

If printed as a whole, each file would look like the below output:

In [23]:
f = open('f056da9c64fbf00a4645ae326e8a4339d015d155.json')
for line in f:
    print(line)

{

    "paper_id": "f056da9c64fbf00a4645ae326e8a4339d015d155",

    "metadata": {

        "title": "SIANN: Strain Identification by Alignment to Near Neighbors",

        "authors": [

            {

                "first": "Samuel",

                "middle": [

                    "S"

                ],

                "last": "Minot",

                "suffix": "",

                "affiliation": {},

                "email": ""

            },

            {

                "first": "Stephen",

                "middle": [

                    "D"

                ],

                "last": "Turner",

                "suffix": "",

                "affiliation": {},

                "email": ""

            },

            {

                "first": "Krista",

                "middle": [

                    "L"

                ],

                "last": "Ternus",

                "suffix": "",

                "affiliation": {},

                "email": ""

            },

            "text": "Relationship of reported value for each program (horizontal axis, log scale) to the empirically-determined Positive Predictive Value (PPV), shown on the vertical axis. While the exact values depend on the test data used, the general values at significant cutoff values (0.8, 0.9, 0.95 PPV) remain relatively constant across different datasets (data not shown).",

            "latex": null,

            "type": "figure"

        },

        "FIGREF1": {

            "text": "The Positive Predictive Value (PPV, vertical axis) is shown for each organism (boxes on right), at each level of known abundance (horizontal axis, see",

            "latex": null,

            "type": "figure"

        },

        "TABREF0": {

            "text": ". Summary of methods for metagenomic classification.",

            "latex": null,

            "type": "table"

        },

        "TABREF1": {

            "text": "Table 2. The abundance of each target organism in each set of simul

## How to load and read a json file

In [24]:
import json

with open('f056da9c64fbf00a4645ae326e8a4339d015d155.json') as f:
  data = json.load(f)

print(data)

{'paper_id': 'f056da9c64fbf00a4645ae326e8a4339d015d155', 'metadata': {'title': 'SIANN: Strain Identification by Alignment to Near Neighbors', 'authors': [{'first': 'Samuel', 'middle': ['S'], 'last': 'Minot', 'suffix': '', 'affiliation': {}, 'email': ''}, {'first': 'Stephen', 'middle': ['D'], 'last': 'Turner', 'suffix': '', 'affiliation': {}, 'email': ''}, {'first': 'Krista', 'middle': ['L'], 'last': 'Ternus', 'suffix': '', 'affiliation': {}, 'email': ''}, {'first': 'Dana', 'middle': ['R'], 'last': 'Kadavy', 'suffix': '', 'affiliation': {}, 'email': ''}]}, 'abstract': [{'text': 'Next-generation sequencing is increasingly being used to study samples composed of mixtures of organisms, such as in clinical applications where the presence of a pathogen at very low abundance may be highly important. We present an analytical method (SIANN: Strain Identification by Alignment to Near Neighbors) specifically designed to rapidly detect a set of target organisms in mixed samples that achieves a hig

In [25]:
data.keys()

dict_keys(['paper_id', 'metadata', 'abstract', 'body_text', 'bib_entries', 'ref_entries', 'back_matter'])

In [26]:
for entry in data['abstract']:
    print(entry['section'],"\n",entry['text'])

Abstract 
 Next-generation sequencing is increasingly being used to study samples composed of mixtures of organisms, such as in clinical applications where the presence of a pathogen at very low abundance may be highly important. We present an analytical method (SIANN: Strain Identification by Alignment to Near Neighbors) specifically designed to rapidly detect a set of target organisms in mixed samples that achieves a high degree of species-and strain-specificity by aligning short sequence reads to the genomes of near neighbor organisms, as well as that of the target. Empirical benchmarking alongside the current state-of-the-art methods shows an extremely high Positive Predictive Value, even at very low abundances of the target organism in a mixed sample. SIANN is available as an Illumina BaseSpace app, as well as through Signature Science, LLC. SIANN results are presented in a streamlined report designed to be comprehensible to the non-specialist user, providing a powerful tool for r

In [27]:
def load_json(filename):
    with open(filename +  ".json") as f:
        data = json.load(f)
    return data

Code to read all json files in the directory in one go (Cite: [Kernel on Kaggle](https://www.kaggle.com/maksimeren/covid-19-literature-clustering))

## Reading all files from HDFS

In [79]:
client_hdfs = InsecureClient('http://madison:31802')
with client_hdfs.read('/cord19dataset/metadata.csv', encoding = 'utf-8') as reader:
    df = pd.read_csv(reader)

df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,xqhn0vbp,1e1286db212100993d03cc22374b624f7caee956,PMC,Airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,PMC140314,12525263.0,no-cc,"BACKGROUND: Rhinovirus, the most common cause ...",2003-01-13,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",BMC Public Health,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
1,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,PMC,Discovering human history from stomach bacteria,10.1186/gb-2003-4-5-213,PMC156578,12734001.0,no-cc,Recent analyses of human pathogens have reveal...,2003-04-28,"Disotell, Todd R",Genome Biol,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
2,le0ogx1s,NaN,PMC,A new recruit for the army of the men of death,10.1186/gb-2003-4-7-113,PMC193621,12844350.0,no-cc,"The army of the men of death, in John Bunyan's...",2003-06-27,"Petsko, Gregory A",Genome Biol,NaN,NaN,False,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
3,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,PMC,Association of HLA class I with severe acute r...,10.1186/1471-2350-4-9,PMC212558,12969506.0,no-cc,BACKGROUND: The human leukocyte antigen (HLA) ...,2003-09-12,"Lin, Marie; Tseng, Hsiang-Kuang; Trejaut, Jean...",BMC Med Genet,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
4,0qaoam29,5b68a553a7cbbea13472721cd1ad617d42b40c26,PMC,A double epidemic model for the SARS propagation,10.1186/1471-2334-3-19,PMC222908,12964944.0,no-cc,BACKGROUND: An epidemic of a Severe Acute Resp...,2003-09-10,"Ng, Tuen Wai; Turinici, Gabriel; Danchin, Antoine",BMC Infect Dis,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...


In [81]:
client_hdfs.content('/cord19dataset')

{'directoryCount': 17,
 'fileCount': 59317,
 'length': 7913288034,
 'quota': -1,
 'spaceConsumed': 23739864102,
 'spaceQuota': -1,
 'typeQuota': {}}

In [84]:
fnames = client_hdfs.list('/cord19dataset')
fnames

['COVID.DATA.LIC.AGMT.pdf',
 'biorxiv_medrxiv',
 'biorxiv_medrxiv.tar.gz',
 'comm_use_subset',
 'cord19_specter_embeddings_2020-04-10',
 'custom_license',
 'json_schema.txt',
 'metadata.csv',
 'metadata.readme',
 'noncomm_use_subset']

In [89]:
fnames = client_hdfs.list('/cord19dataset/biorxiv_medrxiv/biorxiv_medrxiv/pdf_json')
fnames[0:50]

['0015023cc06b5362d332b3baf348d11567ca2fbb.json',
 '00340eea543336d54adda18236424de6a5e91c9d.json',
 '004f0f8bb66cf446678dc13cf2701feec4f36d76.json',
 '00911cf4f99a3d5ae5e5b787675646a743574496.json',
 '00d16927588fb04d4be0e6b269fc02f0d3c2aa7b.json',
 '00eb9220dc8cd351393b6b035323d350f103f8c6.json',
 '0139ea4ca580af99b602c6435368e7fdbefacb03.json',
 '013d9d1cba8a54d5d3718c229b812d7cf91b6c89.json',
 '018fb5e62fbbcae07d57d94d29ac630dcc4dccf9.json',
 '01d162d7fae6aaba8e6e60e563ef4c2fca7b0e18.json',
 '01e3b313e78a352593be2ff64927192af66619b5.json',
 '02201e4601ab0eb70b6c26480cf2bfeae2625193.json',
 '0255ea4b2f26a51a3bfa3bd8f3e1978c82c976d5.json',
 '029c1c588047f1d612a219ee15494d2d19ff7439.json',
 '0313c3faa16cd66d64f31ae37e40fb70695d69fb.json',
 '033ea7af3e6137df652de026f0751ac435327b75.json',
 '03813d8657ba43ea382788caec2d14257b26d8fd.json',
 '03ce432f27c7df6af22b92245a614db2ecb5de5f.json',
 '03ea3a614b56409d3f099c9ad764864293132540.json',
 '03ea9ad47ebe9a599205b99390c45490e6724024.json',


In [ ]:
##It is possible to loop through all the folders and sub folders while using 
##content(<path>,strict=False) != None as a breaking condition`

In [91]:
#There is also this provided snippet to get all files under a given directory
# Get all files under a given folder (arbitrary depth).
import posixpath as psp
fpaths = [
  psp.join(dpath, fname)
  for dpath, _, fnames in client_hdfs.walk('/cord19dataset')
  for fname in fnames
]
len(fpaths)

59317

In [101]:
fpaths_json = list(filter(lambda x: ".json" in x,fpaths))

In [103]:
len(fpaths_json)

59311

## Reading all files from local disk

Since our dataset is not prohibitively large(7-8GB) it is easilty possible to reado it from disk as shown below

In [28]:
all_json = glob.glob(f'{root_path}/**/*.json', recursive=True)
len(all_json)

59311

In [29]:
all_json[0]

'/s/chopin/b/grad/sanketm/cord19dataset/noncomm_use_subset/noncomm_use_subset/pdf_json/e6d882be4961d1bdd7507b4a29d86b650de0895d.json'

In [30]:
_125json = all_json[0:12500]

Let's make a class so that we could refer to the loaded articles easily

In [31]:
class Article:
    
    def __init__(self,filepath):
        with open(filepath) as f:
            metadata = json.load(f)
            self.paper_id = metadata['paper_id']
            self.title = metadata['metadata']['title']
            self.abstract = ""
            self.body_text = ""
            
            #Abstract
            try:
                for entry in metadata['abstract']:
                    self.abstract += str(entry['text'])
            except:
                pass
                
            

            #body_text
            for entry in metadata['body_text']:
                self.body_text += str(entry['text'])


            
    def __repr__(self):
        return f'Article Object with id: {self.paper_id} \n title:{self.title} \n abstract:{self.abstract}'
            
                       

In [32]:
articles = {}
for i in range(5):
    articles[i] = Article(all_json[i])

In [108]:
articles = {}
for i in range(1):
    with client_hdfs.read(fpaths_json[i]) as reader:
        from json import load
        print(load(reader))

{'paper_id': '0015023cc06b5362d332b3baf348d11567ca2fbb', 'metadata': {'title': 'The RNA pseudoknots in foot-and-mouth disease virus are dispensable for genome replication but essential for the production of infectious virus. 2 3', 'authors': [{'first': 'Joseph', 'middle': ['C'], 'last': 'Ward', 'suffix': '', 'affiliation': {}, 'email': ''}, {'first': 'Lidia', 'middle': [], 'last': 'Lasecka-Dykes', 'suffix': '', 'affiliation': {}, 'email': ''}, {'first': 'Chris', 'middle': [], 'last': 'Neil', 'suffix': '', 'affiliation': {}, 'email': ''}, {'first': 'Oluwapelumi', 'middle': [], 'last': 'Adeyemi', 'suffix': '', 'affiliation': {}, 'email': ''}, {'first': 'Sarah', 'middle': [], 'last': '', 'suffix': '', 'affiliation': {}, 'email': ''}, {'first': '', 'middle': [], 'last': 'Gold', 'suffix': '', 'affiliation': {}, 'email': ''}, {'first': 'Niall', 'middle': [], 'last': 'Mclean', 'suffix': '', 'affiliation': {}, 'email': ''}, {'first': 'Caroline', 'middle': [], 'last': 'Wright', 'suffix': '', 'a

In [33]:
articles[0].abstract

'Middle East Respiratory Syndrome coronavirus (MERS-CoV) was first isolated from a patient with severe pneumonia in 2012. The 2015 Korea outbreak of MERS-CoV involved 186 cases, including 38 fatalities. A total of 83% of transmission events were due to five superspreaders, and 44% of the 186 MERS cases were the patients who had been exposed in nosocomial transmission at 16 hospitals. The epidemic lasted for 2 months and the government quarantined 16,993 individuals for 14 days to control the outbreak. This outbreak provides a unique opportunity to fill the gap in our knowledge of MERS-CoV infection. Therefore, in this paper, we review the literature on epidemiology, virology, clinical features, and prevention of MERS-CoV, which were acquired from the 2015 Korea outbreak of MERS-CoV.'

In [34]:
articles[1]

Article Object with id: d0c4236f95ba2b3d5cdc71fde524b26262f05dd3 
 title:Send Orders for Reprints to reprints@benthamscience.net A Brief Review: The Z-curve Theory and its Application in Genome Analysis 
 abstract:In theoretical physics, there exist two basic mathematical approaches, algebraic and geometrical methods, which, in most cases, are complementary. In the area of genome sequence analysis, however, algebraic approaches have been widely used, while geometrical approaches have been less explored for a long time. The Z-curve theory is a geometrical approach to genome analysis. The Z-curve is a three-dimensional curve that represents a given DNA sequence in the sense that each can be uniquely reconstructed given the other. The Z-curve, therefore, contains all the information that the corresponding DNA sequence carries. The analysis of a DNA sequence can then be performed through studying the corresponding Z-curve. The Z-curve method has found applications in a wide range of areas 

# Data Cleaning

Now that we have explored the structure of the dataset, let us focus on our areas of interest as mentioned earlier: namely the Title, Abstract and the body of the text.\
We know that we can access these for a given paper by scanning the metadata for a paper of choice for its title and abstract, and if we need it, using its `sha` column value to get the full text of the paper.

First let us clean the metadata by dropping all NA valued rows for the title, sha and abstract columns.

[Pandas filter rows](https://stackoverflow.com/questions/17071871/how-to-select-rows-from-a-dataframe-based-on-column-values)

In [35]:
bio= meta.loc[meta['source_x'] == 'biorxiv']

In [36]:
print(f'Meta count:{len(meta)} biorxiv papers count: {len(bio)}')

Meta count:51078 biorxiv papers count: 764


In [37]:
bio_clean = bio.drop_duplicates().dropna()

In [38]:
len(bio_clean)

0

Using the above dropna condition doesn't work out,  number of rows in the resulting dataframe is zero.
We found a solution Cite: [stack overflow post](https://stackoverflow.com/questions/39241346/pandas-dropna-on-specify-attribute)

In [39]:
bio_clean = bio.loc[~(bio.sha.isnull())]

In [40]:
len(bio_clean)

678

In [41]:
bio_clean.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
32366,vho70jcx,f056da9c64fbf00a4645ae326e8a4339d015d155,biorxiv,SIANN: Strain Identification by Alignment to N...,10.1101/001727,NaN,NaN,biorxiv,Next-generation sequencing is increasingly bei...,2014-01-10,Samuel Minot; Stephen D Turner; Krista L Ternu...,NaN,NaN,NaN,True,False,biorxiv_medrxiv,https://doi.org/10.1101/001727
32367,i9tbix2v,daf32e013d325a6feb80e83d15aabc64a48fae33,biorxiv,Spatial epidemiology of networked metapopulati...,10.1101/003889,NaN,NaN,biorxiv,An emerging disease is one infectious epidemic...,2014-06-04,Lin WANG; Xiang Li,NaN,NaN,NaN,True,False,biorxiv_medrxiv,https://doi.org/10.1101/003889
32368,62gfisc6,f33c6d94b0efaa198f8f3f20e644625fa3fe10d2,biorxiv,Sequencing of the human IG light chain loci fr...,10.1101/006866,NaN,NaN,biorxiv,Germline variation at immunoglobulin gene (IG)...,2014-07-03,Corey T Watson; Karyn Meltz Steinberg; Tina A ...,NaN,NaN,NaN,True,False,biorxiv_medrxiv,https://doi.org/10.1101/006866
32369,058r9486,4da8a87e614373d56070ed272487451266dce919,biorxiv,Bayesian mixture analysis for metagenomic comm...,10.1101/007476,NaN,NaN,biorxiv,Deep sequencing of clinical samples is now an ...,2014-07-25,Sofia Morfopoulou; Vincent Plagnol,NaN,NaN,NaN,True,False,biorxiv_medrxiv,https://doi.org/10.1101/007476
32370,wich35l7,eccef80cfbe078235df22398f195d5db462d8000,biorxiv,Mapping a viral phylogeny onto outbreak trees ...,10.1101/010389,NaN,NaN,biorxiv,Developing methods to reconstruct transmission...,2014-11-11,Stephen P Velsko; Jonathan E Allen,NaN,NaN,NaN,True,False,biorxiv_medrxiv,https://doi.org/10.1101/010389


In [42]:
print(bio_clean['source_x'].unique(),
bio_clean['pmcid'].unique(),
bio_clean['pubmed_id'].unique(),
bio_clean['Microsoft Academic Paper ID'].unique(),
bio_clean['WHO #Covidence'].unique())

['biorxiv'] [nan] [nan] [nan] [nan]


In [43]:
bio_slim = bio_clean.drop(['Microsoft Academic Paper ID','WHO #Covidence','pubmed_id','pmcid'], axis = 1)

In [44]:
bio_slim = bio_slim.drop(['url','doi','license','full_text_file'],axis =1 )

In [45]:
bio_slim = bio_slim[~bio_slim.has_pdf_parse.isin(['False'])]

In [46]:
bio_slim =bio_slim.loc[~(bio_slim.abstract.isnull())]

As we had seen at the send of section 2.3, all_json contains the paths of all the json files(i.e. the articles) in it.
Now we should load this into a dataframe for it to be queryable

In [47]:
_125json[0]
Article(_125json[0])

Article Object with id: e6d882be4961d1bdd7507b4a29d86b650de0895d 
 title:SEARCH STRATEGY PubMed databases were searched for studies on MERS cases during the 2015 outbreak in the Republic of Korea. The search terms used were combinations of 'Middle East respiratory syndrome' in all fields and 'Korea' in all 
 abstract:Middle East Respiratory Syndrome coronavirus (MERS-CoV) was first isolated from a patient with severe pneumonia in 2012. The 2015 Korea outbreak of MERS-CoV involved 186 cases, including 38 fatalities. A total of 83% of transmission events were due to five superspreaders, and 44% of the 186 MERS cases were the patients who had been exposed in nosocomial transmission at 16 hospitals. The epidemic lasted for 2 months and the government quarantined 16,993 individuals for 14 days to control the outbreak. This outbreak provides a unique opportunity to fill the gap in our knowledge of MERS-CoV infection. Therefore, in this paper, we review the literature on epidemiology, virolog

In [48]:
start_time = datetime.now()
dict_ = {'paper_id': [], 'abstract': [], 'body_text': [], 'authors': [], 'title': []}
for idx, entry in enumerate(_125json):
    if idx % (len(_125json) // 10) == 0:
        print(f'Processing index: {idx} of {len(_125json)}')
    content = Article(entry)
    
    # get metadata information
    meta_data = meta.loc[meta['sha'] == content.paper_id]
    # no metadata, skip this paper
    if len(meta_data) == 0:
        continue
    

    dict_['body_text'].append(content.body_text)
    
       
    # get metadata information
    meta_data = meta.loc[meta['sha'] == content.paper_id]
    
    try:
        # if more than one author
        authors = meta_data['authors'].values[0].split(';')
        if len(authors) > 2:
            # more than 2 authors, may be problem when plotting, so take first 2 append with ...
            dict_['authors'].append(". ".join(authors[:2]) + "...")
        else:
            # authors will fit in plot
            dict_['authors'].append(". ".join(authors))
    except Exception as e:
        # if only one author - or Null valie
        dict_['authors'].append(meta_data['authors'].values[0])
    
    # add the title information, add breaks when needed
    try:
        title = meta_data['title'].values[0], 40
        dict_['title'].append(title)
    # if title was not provided
    except Exception as e:
        dict_['title'].append(meta_data['title'].values[0])
    
       
    dict_['paper_id'].append(meta_data['sha'].values[0])
    dict_['abstract'].append(meta_data['abstract'].values[0])
    
df_covid = pd.DataFrame(dict_, columns=['paper_id', 'abstract', 'body_text', 'authors', 'title'])
df_covid.head()
end_time = datetime.now()
print(f' time taken : {(end_time  - start_time)}')

Processing index: 0 of 12500
Processing index: 1250 of 12500
Processing index: 2500 of 12500
Processing index: 3750 of 12500
Processing index: 5000 of 12500
Processing index: 6250 of 12500
Processing index: 7500 of 12500
Processing index: 8750 of 12500
Processing index: 10000 of 12500
Processing index: 11250 of 12500
 time taken : 0:04:34.872012


In [49]:
print(len(dict_['paper_id']),len(dict_['abstract']),len(dict_['body_text']),len(dict_['authors']),len(dict_['title']))

8811 8811 8811 8811 8811


In [50]:
len(df_covid['abstract'][0].strip().split())

123

In [51]:
df_covid['abstract_word_count'] = df_covid['abstract'].apply(lambda x: len(str(x).strip().split()))
df_covid['body_word_count'] = df_covid['body_text'].apply(lambda x: len(str(x).strip().split()))
df_covid.head(10)

,paper_id,abstract,body_text,authors,title,abstract_word_count,body_word_count
0,e6d882be4961d1bdd7507b4a29d86b650de0895d,Middle East Respiratory Syndrome coronavirus (...,Middle East respiratory syndrome coronavirus (...,"Oh, Myoung-don. Park, Wan Beom...",(Middle East respiratory syndrome: what we lea...,123,3907
1,d0c4236f95ba2b3d5cdc71fde524b26262f05dd3,"In theoretical physics, there exist two basic ...","In theoretical physics, there exist two basic ...","Zhang, Ren. Zhang, Chun-Ting",(A Brief Review: The Z-curve Theory and its Ap...,169,6359
2,d24afd9ee025b53015824be203db539009964fbd,BACKGROUND: The aim of this research was to ev...,In view of the role of thin-section CT scannin...,"Xing, Zhi-Heng. Sun, Xin...",(Thin-section Computed Tomography Detects Long...,326,2937
3,4a3941003ea2673397975ae8bc2536ad59f789e5,INTRODUCTION: The Major Incident Hospital (MIH...,injury. Medical management does not simply inv...,"Marres, G. M. H.. van der Eijk, J....",(Evaluation of admissions to the Major Inciden...,264,4830
4,905e2b35719215760ebed8f3e93fdf6ea8ec9bcf,Mice infected with the neurotropic JHM strain ...,1. Frequently change and/or sterilize gloves t...,"Carbajal, Kevin S.. Weinger, Jason G....",(Surgical Transplantation of Mouse Neural Stem...,137,1300
5,319004f23d1af4357edb2a3862f2619be23a21a6,Diagnostics play a central role in the early d...,► The Middle East respiratory syndrome-coronav...,"Kelly-Cirino, Cassandra. Mazzola, Laura T...",(An updated roadmap for MERS-CoV research and ...,212,3632
6,4971940ab68e4950b28410a208166900e8ba9c07,Despite of the role of domestic dogs as reserv...,Since its initial identification in the late 1...,"Vieira, Flávia V.. Hoffmann, Daniel J....",(Circulation of canine parvovirus among dogs l...,137,1519
7,cae1f2fd5785845caa42d6497361eba46a68f11d,Diversity and plasticity are two hallmarks of ...,Macrophages were first identified by Elie Metc...,"Liu, Yan-Cun. Zou, Xian-Biao...",(Macrophage Polarization in Inflammatory Disea...,111,4608
8,2ffddf5caaef38207b58710a93ee8361518813c9,This commentary offers suggestions for improvi...,"I n the early 20th century, the new field of m...","Kahn, Laura H.","(The need for one health degree programs, 40)",56,1177
9,4bae83e2441c3738d96e49c21c9be0a4c85b4a92,The protective efficacy of DNA plasmids encodi...,Avian infectious bronchitis (IB) is a major di...,"Yan, Fang. Zhao, Yujun...",(Protection of chickens against infectious bro...,179,2987


## Dataset statistics

In [74]:
totalnumberofwords=df_covid['abstract_word_count'].sum()
avg_abstract_len = totalnumberofwords/len(df_covid['abstract_word_count'])
print(f'Total number of words across all abstracts = {totalnumberofwords}. Average abstract length = {avg_abstract_len}')

Total number of words across all abstracts = 1383457. Average abstract length = 157.0147542844172


In [52]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8811 entries, 0 to 8810
Data columns (total 7 columns):
paper_id               8811 non-null object
abstract               7502 non-null object
body_text              8811 non-null object
authors                8543 non-null object
title                  8811 non-null object
abstract_word_count    8811 non-null int64
body_word_count        8811 non-null int64
dtypes: int64(2), object(5)
memory usage: 481.9+ KB


In [53]:
df_covid['abstract'].describe(include='all')

count        7502
unique       7480
top       Unknown
freq           20
Name: abstract, dtype: object

The difference between the unique count and the total count means that either there are some duplicate abstracts or there are blank abstracts that are being counted as the same. 

In [54]:
df_covid['body_text'].describe(include='all')

count                                                  8811
unique                                                 8807
top       In previous reports, workers have characterize...
freq                                                      2
Name: body_text, dtype: object

The difference between the unique count and the total count means that there are some duplicate articles present in the dataset. This coulde be possible if the authors had sent their papers to different journals.

In [55]:
df_covid2 = df_covid.copy()

In [56]:
df_covid2.drop_duplicates(['body_text'],inplace=True)

In [57]:
df_covid2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8807 entries, 0 to 8810
Data columns (total 7 columns):
paper_id               8807 non-null object
abstract               7500 non-null object
body_text              8807 non-null object
authors                8539 non-null object
title                  8807 non-null object
abstract_word_count    8807 non-null int64
body_word_count        8807 non-null int64
dtypes: int64(2), object(5)
memory usage: 550.4+ KB


In [58]:
df_covid2.describe()

,abstract_word_count,body_word_count
count,8807.000000,8807.000000
mean,157.050074,4734.409561
std,97.781201,7540.143374
min,1.000000,1.000000
25%,99.000000,2150.000000
50%,162.000000,3426.000000
75%,220.000000,5343.500000
max,1163.000000,257301.000000


In [59]:
df_covid.describe()

,abstract_word_count,body_word_count
count,8811.000000,8811.000000
mean,157.014754,4735.074793
std,97.787300,7538.883651
min,1.000000,1.000000
25%,99.000000,2151.500000
50%,162.000000,3426.000000
75%,220.000000,5344.000000
max,1163.000000,257301.000000


We now clean the text data so that our key words are clear and we do not have to worry about details we are not concerned about. [ref1](https://towardsdatascience.com/nlp-text-preprocessing-a-practical-guide-and-template-d80874676e79)  [ref2](https://www.geeksforgeeks.org/text-preprocessing-in-python-set-1/)

Firstly By now looking at the abstract columns, we must notice that the text scraped from these articles is not perfect.

For example, many of the abstracts start with the word *Abstract* glued to the beginning of the first word of the real abstract body. 

In [60]:
df_covid2['abstract'] = df_covid2['abstract'].apply(lambda x: str(x).replace('Abstract',''))
df_covid2['abstract'] = df_covid2['abstract'].apply(lambda x: str(x).replace('abstract',''))
df_covid2.head(10)

,paper_id,abstract,body_text,authors,title,abstract_word_count,body_word_count
0,e6d882be4961d1bdd7507b4a29d86b650de0895d,Middle East Respiratory Syndrome coronavirus (...,Middle East respiratory syndrome coronavirus (...,"Oh, Myoung-don. Park, Wan Beom...",(Middle East respiratory syndrome: what we lea...,123,3907
1,d0c4236f95ba2b3d5cdc71fde524b26262f05dd3,"In theoretical physics, there exist two basic ...","In theoretical physics, there exist two basic ...","Zhang, Ren. Zhang, Chun-Ting",(A Brief Review: The Z-curve Theory and its Ap...,169,6359
2,d24afd9ee025b53015824be203db539009964fbd,BACKGROUND: The aim of this research was to ev...,In view of the role of thin-section CT scannin...,"Xing, Zhi-Heng. Sun, Xin...",(Thin-section Computed Tomography Detects Long...,326,2937
3,4a3941003ea2673397975ae8bc2536ad59f789e5,INTRODUCTION: The Major Incident Hospital (MIH...,injury. Medical management does not simply inv...,"Marres, G. M. H.. van der Eijk, J....",(Evaluation of admissions to the Major Inciden...,264,4830
4,905e2b35719215760ebed8f3e93fdf6ea8ec9bcf,Mice infected with the neurotropic JHM strain ...,1. Frequently change and/or sterilize gloves t...,"Carbajal, Kevin S.. Weinger, Jason G....",(Surgical Transplantation of Mouse Neural Stem...,137,1300
5,319004f23d1af4357edb2a3862f2619be23a21a6,Diagnostics play a central role in the early d...,► The Middle East respiratory syndrome-coronav...,"Kelly-Cirino, Cassandra. Mazzola, Laura T...",(An updated roadmap for MERS-CoV research and ...,212,3632
6,4971940ab68e4950b28410a208166900e8ba9c07,Despite of the role of domestic dogs as reserv...,Since its initial identification in the late 1...,"Vieira, Flávia V.. Hoffmann, Daniel J....",(Circulation of canine parvovirus among dogs l...,137,1519
7,cae1f2fd5785845caa42d6497361eba46a68f11d,Diversity and plasticity are two hallmarks of ...,Macrophages were first identified by Elie Metc...,"Liu, Yan-Cun. Zou, Xian-Biao...",(Macrophage Polarization in Inflammatory Disea...,111,4608
8,2ffddf5caaef38207b58710a93ee8361518813c9,This commentary offers suggestions for improvi...,"I n the early 20th century, the new field of m...","Kahn, Laura H.","(The need for one health degree programs, 40)",56,1177
9,4bae83e2441c3738d96e49c21c9be0a4c85b4a92,The protective efficacy of DNA plasmids encodi...,Avian infectious bronchitis (IB) is a major di...,"Yan, Fang. Zhao, Yujun...",(Protection of chickens against infectious bro...,179,2987


Now let us remove all punctuation from text and change it to lowercase

# Data Preprocessing

In [61]:
start_time = datetime.now()
# import nltk 
import string 
import re 
end_time = datetime.now()
print(f'imported text preprocessing libraries, time taken:{end_time-start_time}')

imported text preprocessing libraries, time taken:0:00:00.000157


In [62]:
df_covid2['abstract'] = df_covid2['abstract'].apply(lambda x: str(x).lower())
df_covid2.head()

,paper_id,abstract,body_text,authors,title,abstract_word_count,body_word_count
0,e6d882be4961d1bdd7507b4a29d86b650de0895d,middle east respiratory syndrome coronavirus (...,Middle East respiratory syndrome coronavirus (...,"Oh, Myoung-don. Park, Wan Beom...",(Middle East respiratory syndrome: what we lea...,123,3907
1,d0c4236f95ba2b3d5cdc71fde524b26262f05dd3,"in theoretical physics, there exist two basic ...","In theoretical physics, there exist two basic ...","Zhang, Ren. Zhang, Chun-Ting",(A Brief Review: The Z-curve Theory and its Ap...,169,6359
2,d24afd9ee025b53015824be203db539009964fbd,background: the aim of this research was to ev...,In view of the role of thin-section CT scannin...,"Xing, Zhi-Heng. Sun, Xin...",(Thin-section Computed Tomography Detects Long...,326,2937
3,4a3941003ea2673397975ae8bc2536ad59f789e5,introduction: the major incident hospital (mih...,injury. Medical management does not simply inv...,"Marres, G. M. H.. van der Eijk, J....",(Evaluation of admissions to the Major Inciden...,264,4830
4,905e2b35719215760ebed8f3e93fdf6ea8ec9bcf,mice infected with the neurotropic jhm strain ...,1. Frequently change and/or sterilize gloves t...,"Carbajal, Kevin S.. Weinger, Jason G....",(Surgical Transplantation of Mouse Neural Stem...,137,1300


In [63]:
df_covid2['body_text'] = df_covid2['body_text'].apply(lambda x: str(x).lower())
df_covid2.head()

,paper_id,abstract,body_text,authors,title,abstract_word_count,body_word_count
0,e6d882be4961d1bdd7507b4a29d86b650de0895d,middle east respiratory syndrome coronavirus (...,middle east respiratory syndrome coronavirus (...,"Oh, Myoung-don. Park, Wan Beom...",(Middle East respiratory syndrome: what we lea...,123,3907
1,d0c4236f95ba2b3d5cdc71fde524b26262f05dd3,"in theoretical physics, there exist two basic ...","in theoretical physics, there exist two basic ...","Zhang, Ren. Zhang, Chun-Ting",(A Brief Review: The Z-curve Theory and its Ap...,169,6359
2,d24afd9ee025b53015824be203db539009964fbd,background: the aim of this research was to ev...,in view of the role of thin-section ct scannin...,"Xing, Zhi-Heng. Sun, Xin...",(Thin-section Computed Tomography Detects Long...,326,2937
3,4a3941003ea2673397975ae8bc2536ad59f789e5,introduction: the major incident hospital (mih...,injury. medical management does not simply inv...,"Marres, G. M. H.. van der Eijk, J....",(Evaluation of admissions to the Major Inciden...,264,4830
4,905e2b35719215760ebed8f3e93fdf6ea8ec9bcf,mice infected with the neurotropic jhm strain ...,1. frequently change and/or sterilize gloves t...,"Carbajal, Kevin S.. Weinger, Jason G....",(Surgical Transplantation of Mouse Neural Stem...,137,1300


In [64]:
str.maketrans?

In [65]:
print("Punctuated:\n",df_covid2['abstract'][1],'\n\n\n')
translator = str.maketrans(' ',' ',string.punctuation)
print("Cleaned:\n",df_covid2['abstract'][1].translate(translator))

Punctuated:
 in theoretical physics, there exist two basic mathematical approaches, algebraic and geometrical methods, which, in most cases, are complementary. in the area of genome sequence analysis, however, algebraic approaches have been widely used, while geometrical approaches have been less explored for a long time. the z-curve theory is a geometrical approach to genome analysis. the z-curve is a three-dimensional curve that represents a given dna sequence in the sense that each can be uniquely reconstructed given the other. the z-curve, therefore, contains all the information that the corresponding dna sequence carries. the analysis of a dna sequence can then be performed through studying the corresponding z-curve. the z-curve method has found applications in a wide range of areas in the past two decades, including the identifications of protein-coding genes, replication origins, horizontally-transferred genomic islands, promoters, translational start sides and isochores, as wel

In [66]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [67]:
start_time= datetime.now()
df_covid2['abstract'] = df_covid2['abstract'].apply(lambda x: str(x).translate(translator))
df_covid2.head()
end_time=datetime.now()
print(f'removed all punctuation from all abstracts. time taken:{end_time-start_time}')

removed all punctuation from all abstracts. time taken:0:00:00.588561


In [68]:
start_time= datetime.now()
df_covid2['body_text'] = df_covid2['body_text'].apply(lambda x: str(x).translate(translator))
df_covid2.head()
end_time=datetime.now()
print(f'removed all punctuation from all body_texts. time taken:{end_time-start_time}')

removed all punctuation from all body_texts. time taken:0:00:26.866739


In [69]:
abstracts = df_covid2.drop(["paper_id","body_text", "abstract_word_count", "body_word_count", "authors", "title"], axis=1)
abstracts.head()

,abstract
0,middle east respiratory syndrome coronavirus m...
1,in theoretical physics there exist two basic m...
2,background the aim of this research was to eva...
3,introduction the major incident hospital mih i...
4,mice infected with the neurotropic jhm strain ...


In [70]:
bodytexts = df_covid2.drop(["paper_id","abstract", "abstract_word_count", "body_word_count", "authors", "title"], axis=1)
bodytexts.head()

,body_text
0,middle east respiratory syndrome coronavirus m...
1,in theoretical physics there exist two basic m...
2,in view of the role of thinsection ct scanning...
3,injury medical management does not simply invo...
4,1 frequently change andor sterilize gloves thr...


In [71]:
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords

#stopwords = nltk.corpus.stopwords.words('english')

def normalize_document(doc):

    # lower case and remove special characters\whitespaces

    doc = str(doc).strip()
    doc = str(doc).lower()
    
    translator = str.maketrans(' ',' ',string.punctuation)  #where string.punctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    doc = str(doc).translate(translator)

    # tokenize document

    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(doc) 
    filtered_text = [word for word in word_tokens if word not in stop_words] 
    return filtered_text

In [72]:
abstracts = abstracts['abstract'].apply(normalize_document(x))

NameError: name 'x' is not defined

## Forming n-grams

[ref](https://stackoverflow.com/questions/21844546/forming-bigrams-of-words-in-list-of-sentences-with-python)

In [ ]:
from nltk import word_tokenize
from nltk.util import ngrams

In [ ]:
for i in ngrams(abstracts['abstract'][0].split(" "),2):
    print(i)

# Word Cloud of Abstracts

In this section we make a word cloud of the abstracts we have got. [ref](https://www.datacamp.com/community/tutorials/wordcloud-python)

In [ ]:
import wordcloud as wc

In [ ]:
# Start with one review:

text = abstracts['abstract'][0]

# Create and generate a word cloud image:
wordcloud = wc.WordCloud().generate(text)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
## Target Column:
print(f"Total number of abstracts: {len(abstracts['abstract'])}")
print(f"Number of nan abstracts: {abstracts['abstract'].value_counts()['nan']}")

In [ ]:
abstracts = abstracts.loc[~(abstracts['abstract'] == 'nan')]

In [ ]:
## Target Column:
print(f"Total number of abstracts: {len(abstracts['abstract'])}")

In [ ]:
start_time = datetime.now()
wordcloud2 = wc.WordCloud(width=1920,height=1080).generate(' '.join(abstracts['abstract']))
end_time=datetime.now()
print(f'generated wordclouds from all {len(abstracts)} abstracts. time taken:{end_time-start_time}')

In [ ]:
plot = plt.figure(figsize=(60, 60))
plt.imshow(wordcloud2)
plt.axis("off")
plt.show()

# Forming word Vectors

In [ ]:
from sklearn import Tf

# Dimensionality Reduction

# Clustering 

[Ref - Scikit learn](https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html#sphx-glr-auto-examples-text-plot-document-clustering-py)

# Visualization and stuff

# References

1. [COVID-19 Open Research Dataset Challenge (CORD-19)](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge/)
2. [How to build a topic-based search engine](https://www.smithinst.co.uk/insights/build-topic-based-search-engine/)
3. W. Buntine et al., "A Scalable Topic-Based Open Source Search Engine," IEEE/WIC/ACM International Conference on Web Intelligence (WI'04), Beijing, China, 2004, pp. 228-234.
4. [Semantic Topic Modeling for Search Queries at Google](https://gofishdigital.com/semantic-topic-modeling/)
5. GRANT, C.; PAZHAYIDAM GEORGE, C.; KANJILAL, V.; NIRKHIWALE, S.; WILSON, J.; WANG, D.. A Topic-Based Search, Visualization, and Exploration System. Florida Artificial Intelligence Research Society Conference, North America, apr. 2015. Available at: <https://www.aaai.org/ocs/index.php/FLAIRS/FLAIRS15/paper/view/10445/10360>. Date accessed: 25 Apr. 2020.